In [ ]:
# Project
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
# Path to your WebDriver executable
webdriver_path = '/Users/kumarmayank/chromedriver_mac_arm64'

# Path to the Tampermonkey extension CRX file
tampermonkey_path = '/Users/kumarmayank/dhdgffkkebhmkfjojejmpbldmpobfkfo-4.19.0-Crx4Chrome.com.crx'

# Search query for Google Images
# search_query = 'cats'
search_query = input("Enter the keywords to search:")
# Number of images to scrape
#num_images_to_scrape = 10
num_images_to_scrape = int(input("Enter the number of images:"))

# Create ChromeOptions instance and add the Tampermonkey extension (Correction: Open the existing chrome )
chrome_options = Options()
chrome_options.add_extension(tampermonkey_path)

# Create a new instance of the WebDriver with the specified options
driver = webdriver.Chrome(webdriver_path, options=chrome_options)

# Navigate to the Google Images website
url="https://www.google.com/search?q=" + search_query + "&tbm=isch"
driver.get(url)
# driver.get('https://www.google.com/imghp')

# Find the search input element and enter the search query
search_input = driver.find_element(By.CSS_SELECTOR, 'input[name="q"]')
search_input.send_keys(search_query)
search_input.submit()

# Wait for the page to load
time.sleep(5)

# TamperMoney Code
# Activate the Tampermonkey script
script_id = 'Super Google Images'  # Replace with the actual Tampermonkey script ID
script_url = 'https://raw.githubusercontent.com/FarisHijazi/SuperGoogleImages/master/SuperGoogleImages.user.js'
# Replace with the URL of your script

driver.execute_script(f"""
    var script = document.createElement('script');
    script.src = '{script_url}';
    script.setAttribute('id', '{script_id}');
    document.head.appendChild(script);
""")

# Scroll down to load more images
for _ in range(3):
    driver.execute_script('window.scrollBy(0, document.body.scrollHeight);')
    time.sleep(2)

# Find all the image elements
image_elements = driver.find_elements(By.CSS_SELECTOR, 'img.rg_i')

# Scrape the image URLs
image_urls = []
for i, image_element in enumerate(image_elements):
    # Click on each image to get the full-sized version
    if i >= num_images_to_scrape:
        break
    try:
        image_element.click()
        time.sleep(2)
        image_urls.append(driver.find_element_by_css_selector('img.n3VNCb').get_attribute('src'))
    except Exception as e:
        print(f'Error scraping image {i + 1}: {str(e)}')

# Print the scraped image URLs
for i, url in enumerate(image_urls):
    print(f'Image {i + 1}: {url}')

# Close the WebDriver
driver.quit()